# งานที่ 3: ใช้ Amazon Bedrock สำหรับการตอบคำถาม

ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้วิธีใช้โมเดล Bedrock Titan เพื่อให้การตอบกลับเป็นข้อมูลเมื่อได้รับคำถาม โดยส่งคำขอพร้อมบริบทที่เกี่ยวข้องทั้งหมดไปยังโมเดลและคาดหวังการตอบกลับ ตอบสนองต่อความท้าทายในการให้โมเดลส่งคำตอบเป็นข้อเท็จจริงสำหรับคำถาม โดยไม่จำเป็นต้องเตรียมและจัดทำเอกสารดัชนีล่วงหน้า

โน้ตบุ๊กนี้จำลองสิ่งที่ **Retrieval-Augmented Generation (RAG)** สามารถทำอะไรได้บ้าง แต่ไม่ได้ใช้ RAG จริงๆ วิธีการนี้ใช้ได้กับเอกสารสั้นๆ หรือแอปพลิเคชันแบบ single-ton โดยอาจไม่มีการปรับขนาดตามคำตอบของคำถามระดับองค์กร ซึ่งเอกสารองค์กรขนาดใหญ่ไม่สามารถใส่เข้ากับพรอมต์คำสั่งที่ส่งไปยังโมเดลได้

**การตอบคำถาม (QA)** เป็นงานสำคัญที่เกี่ยวข้องกับการแยกคำตอบสำหรับคำถามข้อเท็จจริงที่เกิดขึ้นในภาษาธรรมชาติ โดยทั่วไป ระบบ QA จะประมวลผลคำถามกับฐานความรู้ที่มีข้อมูลซึ่งที่มีหรือไม่มีโครงสร้าง และสร้างการตอบกลับที่มีข้อมูลที่ถูกต้อง การรับประกันความแม่นยำสูงเป็นกุญแจสำคัญในการพัฒนาระบบตอบคำถามที่มีประโยชน์ ไว้วางใจได้ และเชื่อถือได้ โดยเฉพาะในกรณีการใช้งานในระดับองค์กร


## สถานการณ์

คุณลองสร้างโมเดลสถานการณ์ที่ AnyCompany ที่คุณถามคำถามโมเดลเพื่อตอบคำถามที่ให้ข้อมูลเกี่ยวกับการเปลี่ยนยางสำหรับรถโมเดลเฉพาะที่ตนผลิต ก่อนอื่นคุณต้องทำการคิวรีโมเดลโดยใช้วิธี “zero shot” เพื่อดูว่าสามารถให้คำตอบที่เกี่ยวข้องตามข้อมูลที่ได้ฝึกมาหรือไม่

อย่างไรก็ตาม คุณรู้ว่าโมเดลนี้ดูเหมือนจะมีคำตอบทั่วไปที่ “ไม่จริง” เยอะ ตามที่ได้พิสูจน์เมื่อลองโมเดลยานพาหนะปลอมและได้รับการตอบกลับที่คล้ายกัน นี่หมายความว่าจำเป็นต้องเพิ่มการฝึกด้วยคู่มือยานพาหนะจริงของ Example Company เพื่อระบุรายละเอียดเกี่ยวกับยางสำหรับรถแต่ละรุ่น

ในแล็บนี้ คุณจำลองวิธีการ “การสร้างการดึงข้อมูลเพิ่มขึ้น” (RAG) โดยไม่มีข้อมูลภายนอก คุณให้ข้อความที่ตัดตอนมาโดยละเอียดเกี่ยวกับวิธีการเปลี่ยนยางในรถยนต์ AnyCompany Model Z คุณจะทดสอบว่าตอนนี้โมเดลสามารถให้คำตอบที่กำหนดเองและถูกต้องโดยใช้ประโยชน์จากเนื้อหาตัวอย่างในบริบทนี้หรือไม่

## งานที่ 3.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## งานที่ 3.2: คำถามและคำตอบด้วยความรู้เกี่ยวกับโมเดล
ในส่วนนี้ เราพยายามใช้โมเดลที่จัดทำโดยบริการ Bedrock เพื่อตอบคำถามตามความรู้ที่ได้รับในระหว่างขั้นตอนการฝึก

ในงานนี้ ให้คุณใช้วิธี invoke_model () ของไคลเอ็นต์ Amazon Bedrock พารามิเตอร์ที่จำเป็นในการใช้วิธีนี้คือ modelId ซึ่งแทนโมเดล Amazon Bedrock ARN และ Body ซึ่งเป็นพรอมต์คำสั่งสำหรับงานของคุณ

พรอมต์คำสั่ง Body จะเปลี่ยนแปลงโดยขึ้นอยู่กับผู้ให้บริการโมเดลพื้นฐานที่คุณเลือกใช้ คุณลองดูรายละเอียดด้านล่าง

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

คุณพยายามใช้โมเดลที่ให้บริการโดยบริการ Bedrock เพื่อตอบคำถามตามความรู้ที่ได้รับในระหว่างขั้นตอนการฝึก

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## งานที่ 3.3: เรียกโมเดลโดยส่งผ่าน Body ของ JSON เพื่อสร้างการตอบกลับ

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


โมเดลนี้ให้คำตอบเกี่ยวกับกระบวนการเปลี่ยนยางที่แบนของรถ โดยคำอธิบายเดียวกันนี้อาจสามารถใช้ได้กับรถยนต์ทุกคัน น่าเสียดายที่นี่ไม่ใช่คำตอบที่ถูกต้องสำหรับ AnyCompany AC8 ซึ่งไม่มียางสำรอง สิ่งนี้เกิดขึ้นได้เนื่องจากโมเดลได้รับการฝึกเกี่ยวกับข้อมูลที่มีคำแนะนำเกี่ยวกับการเปลี่ยนยางรถยนต์

อีกตัวอย่างหนึ่งของปัญหานี้สามารถมองเห็นได้โดยลองถามคำถามเดียวกันเรื่องยี่ห้อและโมเดลของรถที่เป็นของปลอม โดยสมมติว่าเป็น Amazon Tirana

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

เมื่อประเมินจากคำถามพรอมต์คำสั่งแล้ว โมเดลจะไม่สามารถให้คำตอบที่สมจริงได้

หากต้องการแก้ไขปัญหานี้และให้โมเดลตอบตามคำสั่งที่เฉพาะเจาะจงที่ถูกต้องสำหรับโมเดลรถของคุณ คุณสามารถเพิ่มความรู้ของโมเดลได้ทันทีโดยให้ฐานความรู้เพิ่มเติมเป็นส่วนหนึ่งของพรอมต์คำสั่ง

มาดูกันว่าคุณจะใช้สิ่งนี้เพื่อปรับปรุงแอปพลิเคชันของคุณได้อย่างไรบ้าง

สมมติว่าต่อไปนี้เป็นข้อความที่ตัดตอนมาจากคู่มือของ AnyCompany AC8 (ในความเป็นจริง นี่ไม่ใช่คู่มือจริง แต่ให้สมมติกันว่าเป็นของจริง) เอกสารนี้สั้นพอที่จะพอดีกับหน้าต่างบริบทของ Titan Large ทั้งหมด

```plain
ยางรถและความดันลมยาง:

ยางรถทำมาจากยางสีดำและติดตั้งบนล้อของยานพาหนะของคุณ ยางรถจะให้การยึดเกาะที่จำเป็นสำหรับการขับขี่ทางโค้งและการเบรก ปัจจัยสำคัญสองประการที่ควรพิจารณาคือ ความดันลมยางและการสึกหรอของยางเนื่องจากอาจส่งผลต่อประสิทธิภาพและการควบคุมรถของคุณ

จะหาแรงดันยางที่แนะนำได้ที่ไหน:

คุณสามารถดูข้อมูลจำเพาะความดันลมยางที่แนะนำได้บนฉลากอัตราการเติมน้ำมันที่อยู่บนเสา B ด้านคนขับของรถ หรือคุณสามารถอ้างอิงจากคู่มือรถสำหรับข้อมูลนี้ ความดันลมยางที่แนะนำอาจแตกต่างกันไปขึ้นอยู่กับความเร็วและจำนวนผู้โดยสารหรือน้ำหนักบรรทุกสูงสุดในรถ

การเติมลมยางใหม่:

เมื่อตรวจสอบความดันลมยาง สิ่งสำคัญคือต้องทำเมื่อยางเย็น ซึ่งหมายความว่าให้ยานพาหนะจอดนิ่งเป็นเวลาอย่างน้อยสามชั่วโมงเพื่อให้แน่ใจว่ายางอยู่ในอุณหภูมิเดียวกับอุณหภูมิแวดล้อม

วิธีเติมลมยาง:

    ตรวจสอบความดันลมยางที่แนะนำสำหรับยานพาหนะของคุณ
    ทำตามคำแนะนำที่มีบนปั๊มลม และเติมลมยางด้วยแรงดันที่ถูกต้อง
    ในจอแสดงผลตรงกลางของยานพาหนะของคุณ ให้เปิดแอป “สถานะรถยนต์”
    ไปที่แท็บ “ความดันลมยาง”
    กดตัวเลือก “ปรับเทียบความดัน” และยืนยันการกระทำ
    ขับรถสักสองสามนาทีด้วยความเร็วสูงกว่า 30 กม./ชม. เพื่อปรับเทียบความดันลมยาง

หมายเหตุ: ในบางกรณี อาจจำเป็นต้องขับรถนานกว่า 15 นาทีเพื่อให้สัญลักษณ์เตือนหรือข้อความที่เกี่ยวข้องกับความดันลมยางหายไป หากคำเตือนยังคงอยู่ ให้ปล่อยให้ยางเย็นลงและทำซ้ำขั้นตอนข้างต้น

ยางแบน:

หากยางแบนขณะขับรถ คุณสามารถปะยางชั่วคราวและเติมลงยางใหม่โดยใช้ชุดซ่อมยางพกพาได้ โดยปกติชุดนี้จะถูกเก็บไว้ใต้ที่บุรองบริเวณที่วางกระเป๋าเดินทางในรถ

คำแนะนำในการใช้ชุดซ่อมยางพกพา:

    เปิดประตูท้ายหรือท้ายรถ
    ยกที่บุรองบริเวณที่วางกระเป๋าเดินทางขึ้นเพื่อเข้าถึงชุดซ่อมยางพกพา
    ทำตามคำแนะนำที่ให้มาพร้อมกับชุดซ่อมยางพกพาเพื่อทำการปะยาง
    หลังจากใช้ชุดซ่อมแล้ว ตรวจสอบว่าได้วางกลับเข้าตำแหน่งเดิมอย่างแน่นหนา
    ติดต่อ Rivesla หรือบริการที่เหมาะสมเพื่อขอความช่วยเหลือในการกำจัดและเปลี่ยนขวดกาวที่ใช้แล้ว

โปรดทราบว่าชุดซ่อมยางพกพาเป็นวิธีแก้ปัญหาชั่วคราวและได้รับการออกแบบมาเพื่อให้คุณสามารถขับรถได้สูงสุด 10 นาที หรือ 8 กม. (ไม่ว่าข้อใดจะเกิดขึ้นก่อน) ด้วยความเร็วสูงสุด 80 กม./ชม. ขอแนะนำให้เปลี่ยนยางที่รั่วหรือให้ช่างซ่อมมืออาชีพโดยเร็วที่สุด
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### ตอนนี้ ให้ส่งข้อความที่ตัดตอนมาทั้งหมดไปยังโมเดลพร้อมกับคำถาม

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### งานที่ 3.4: เรียกโมเดลผ่าน boto3 เพื่อสร้างการตอบกลับ

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

เนื่องจากโมเดลใช้เวลาสักครู่ในการทำความเข้าใจบริบทและสร้างคำตอบที่เกี่ยวข้องสำหรับคุณ สิ่งนี้อาจนำไปสู่ประสบการณ์ของผู้ใช้ที่ไม่ดีเนื่องจากลูกค้าต้องรอการตอบกลับเป็นเวลาสองสามวินาที

Bedrock ยังรองรับความสามารถในการสตรีมมิ่งที่บริการสร้างเอาต์พุตในขณะที่โมเดลสร้างโทเค็น นี่คือตัวอย่างของวิธีที่คุณสามารถนำไปใช้ได้

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

การตอบกลับจะประกอบไปด้วยคำแนะนำที่มีการสรุปทีละขั้นตอนเกี่ยวกับวิธีการเปลี่ยนยาง 

ตอนนี้คุณได้เรียนรู้แล้วว่า คุณสามารถใช้ประโยชน์จาก Recrieval Augmented Generation (RAG) หรือกระบวนการ Augmentation เพื่อสร้างการตอบกลับที่คัดสรรมาแล้ว ซึ่งปรับให้เหมาะกับบริบทและข้อมูลที่เฉพาะเจาะจง

### ลองด้วยตัวเอง
- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้และไปต่อยัง **งานที่ 4**